In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE37837"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE37837"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE37837.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE37837.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE37837.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Check if gene expression data is available
# From the background information, we see this dataset contains genome-wide expression analysis
# using Agilent whole human genome oligo microarray
is_gene_available = True

# 2. Determine variable availability and create conversion functions

# 2.1 Trait Data
# Here, endometriosis status can be determined from the 'tissue' field (row 2)
# Looking at the unique values, we can see "Autologous_eutopic" vs "Endometrioma_ectopic"
trait_row = 2

def convert_trait(value):
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    if isinstance(value, str) and "Endometrioma_ectopic" in value:
        return 1  # Endometriotic tissue
    elif isinstance(value, str) and "Autologous_eutopic" in value:
        return 0  # Normal endometrial tissue
    return None

# 2.2 Age Data
# Age is available in row 0
age_row = 0

def convert_age(value):
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        if isinstance(value, str):
            # Extract numeric age value
            age = int(value.split()[0])
            return age  # Return as continuous value
    except:
        pass
    return None

# 2.3 Gender Data
# All samples are from females according to row 1
# Since this is a constant feature (only one value), we'll mark it as not available
gender_row = None

def convert_gender(value):
    # Not needed since gender is not variable in this dataset, but included for completeness
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    if isinstance(value, str) and "female" in value.lower():
        return 0
    elif isinstance(value, str) and "male" in value.lower():
        return 1
    return None

# 3. Save metadata using the validate_and_save_cohort_info function
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # The Sample Characteristics Dictionary was provided in the previous step
    sample_chars = {0: ['age (y): 29', 'age (y): 40', 'age (y): 33', 'age (y): 45', 'age (y): 24', 'age (y): 38', 'age (y): 28', 'age (y): 25', 'age (y): 31', 'age (y): 37', 'age (y): 30', 'age (y): 34'], 1: ['gender: female (fertile)'], 2: ['tissue: Autologous_eutopic', 'tissue: Endometrioma_ectopic'], 3: ['subject id: E17', 'subject id: E20', 'subject id: E23', 'subject id: E26', 'subject id: E31', 'subject id: E32', 'subject id: E33', 'subject id: E40', 'subject id: E43', 'subject id: E48', 'subject id: E49', 'subject id: E52', 'subject id: E56', 'subject id: E57', 'subject id: E68', 'subject id: E70', 'subject id: E73', 'subject id: E75'], 4: ['menstrual phase: Proliferative', 'menstrual phase: Secretory'], 5: ['endometrioma severity stage: Severe (stage 4)', 'endometrioma severity stage: Moderate (stage 3)'], 6: ['parity: Pregnancy_1; live offspriing_1', 'parity: Pregnancy_6; live offspriing_6', 'parity: Pregnancy_3; live offspriing_3', 'parity: Pregnancy_3; live offspriing_2', 'parity: Pregnancy_2; live offspriing_1', 'parity: Pregnancy_4; live offspriing_2', 'parity: Pregnancy_2; live offspriing_2', 'parity: Pregnancy_4; live offspriing_4']}
    
    # First, let's create a dataframe where rows are the feature indices and columns are the sample IDs
    # Start with an empty list to hold the sample IDs
    sample_ids = []
    # Extract subject IDs from row 3
    for sample_id_str in sample_chars[3]:
        if ':' in sample_id_str:
            sample_id = sample_id_str.split(':', 1)[1].strip()
            sample_ids.append(sample_id)
    
    # Create an empty dataframe with rows as feature indices and columns as sample IDs
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_ids)
    
    # Now fill the dataframe
    # For each feature row and sample, determine the appropriate value
    for row_idx, values in sample_chars.items():
        for sample_id in sample_ids:
            # For each sample, find the most appropriate value
            # For now, we'll just use the first value in the list
            if values:
                clinical_data.loc[row_idx, sample_id] = values[0]
    
    # Use geo_select_clinical_features to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    for key, value in preview.items():
        print(f"{key}: {value}")
    
    # Save the extracted clinical features to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# Observing the gene identifiers in the data
# Based on the preview, we see identifiers like 'A_23_P100001' which are Agilent microarray probe IDs, 
# not standard human gene symbols
# These IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# Examine the gene annotation to identify mapping columns
# From the preview, we can see that 'ID' in the gene annotation corresponds to the probe identifiers
# 'GENE_SYMBOL' contains the human gene symbols we want to map to

# 1. Extract the mapping between probe IDs and gene symbols using the get_gene_mapping function
probe_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_symbol_col)

# 2. Print a sample of the mapping to verify
print("Gene mapping preview (probe ID to gene symbol):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 4. Preview the first few rows of the gene expression data
print("\nGene expression data preview after mapping:")
print(gene_data.head())

# 5. Report the shape of the gene expression data
print(f"\nGene expression data shape: {gene_data.shape}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Load clinical features from the previously saved file
clinical_features_df = pd.read_csv(out_clinical_data_file, index_col=0)

# Create a mapping between GSM IDs and subject IDs from the SOFT file
gsm_to_subject_mapping = {}

# Extract the mapping from the SOFT file
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith('!Sample_geo_accession'):
            gsm_id = line.strip().split('=')[1].strip('"')
        elif line.startswith('!Sample_source_name_ch1'):
            if 'E' in line:
                # Extract subject ID (usually in format "subject id: E##")
                subject_id = 'E' + line.split('E')[1].split()[0]
                gsm_to_subject_mapping[gsm_id] = subject_id

# If mapping was created successfully, transform clinical features to align with GSM IDs
if gsm_to_subject_mapping:
    # Create a new clinical dataframe using GSM IDs as index
    new_clinical_df = pd.DataFrame(index=normalized_gene_data.columns)
    
    # Map trait values from subject IDs to GSM IDs
    for gsm_id, subject_id in gsm_to_subject_mapping.items():
        if gsm_id in new_clinical_df.index and subject_id in clinical_features_df.columns:
            for feature in clinical_features_df.index:
                if feature == trait:
                    new_clinical_df.loc[gsm_id, feature] = clinical_features_df.loc[feature, subject_id]
                elif feature == 'Age':
                    new_clinical_df.loc[gsm_id, feature] = clinical_features_df.loc[feature, subject_id]
    
    clinical_features_df = new_clinical_df.T  # Transpose to get features as rows
else:
    # If mapping failed, create clinical data from scratch based on SOFT file information
    # Extract tissue and age information from the SOFT file
    tissue_dict = {}
    age_dict = {}
    
    with gzip.open(soft_file, 'rt') as f:
        current_gsm = None
        for line in f:
            line = line.strip()
            if line.startswith('!Sample_geo_accession'):
                current_gsm = line.split('=')[1].strip('"')
            elif current_gsm and line.startswith('!Sample_characteristics_ch1'):
                value = line.split('=')[1].strip('"')
                if 'tissue:' in value:
                    tissue_dict[current_gsm] = 1 if "Endometrioma_ectopic" in value else 0
                elif 'age (y):' in value:
                    try:
                        age = int(value.split(':')[1].strip().split()[0])
                        age_dict[current_gsm] = age
                    except (IndexError, ValueError):
                        pass
    
    # Create clinical dataframe with GSM IDs as index
    new_clinical_df = pd.DataFrame(index=normalized_gene_data.columns)
    new_clinical_df[trait] = new_clinical_df.index.map(tissue_dict)
    new_clinical_df['Age'] = new_clinical_df.index.map(age_dict)
    
    # Transpose to get features as rows
    clinical_features_df = new_clinical_df.T

# Now link the clinical and genetic data
linked_data = pd.concat([clinical_features_df, normalized_gene_data], axis=0)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from eutopic and ectopic endometrial tissues from women with endometriosis."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create a dictionary to store GSM to tissue type mapping
gsm_to_tissue = {}
gsm_to_age = {}

# Extract tissue type and age for each GSM ID directly from the SOFT file
with gzip.open(soft_file, 'rt') as f:
    current_gsm = None
    for line in f:
        line = line.strip()
        if line.startswith('!Sample_geo_accession'):
            current_gsm = line.split('=')[1].strip('"')
        elif current_gsm and line.startswith('!Sample_characteristics_ch1'):
            value = line.split('=')[1].strip('"')
            if 'tissue:' in value:
                gsm_to_tissue[current_gsm] = 1 if "Endometrioma_ectopic" in value else 0
            elif 'age (y):' in value:
                try:
                    age = int(value.split(':')[1].strip().split()[0])
                    gsm_to_age[current_gsm] = age
                except (IndexError, ValueError):
                    pass

# Print sample of mappings to verify data extraction
print(f"Sample of tissue mappings: {list(gsm_to_tissue.items())[:5]}")
print(f"Sample of age mappings: {list(gsm_to_age.items())[:5]}")
print(f"Total GSMs with tissue data: {len(gsm_to_tissue)}")
print(f"Total GSMs with age data: {len(gsm_to_age)}")

# Create clinical data as a DataFrame with appropriate structure for linking
# Using the gene expression data column names as sample IDs
clinical_data = pd.DataFrame(index=[trait, 'Age'])

# Add data for each sample
for gsm in normalized_gene_data.columns:
    if gsm in gsm_to_tissue:
        clinical_data.at[trait, gsm] = gsm_to_tissue[gsm]
    if gsm in gsm_to_age:
        clinical_data.at['Age', gsm] = gsm_to_age[gsm]

# Verify clinical data content
print("Clinical data shape:", clinical_data.shape)
print("Clinical data sample:")
print(clinical_data.iloc[:, :5])

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = pd.concat([clinical_data, normalized_gene_data])
print("Linked data shape:", linked_data.shape)

# Print a quick check of the trait column
trait_values = clinical_data.loc[trait]
print(f"Number of samples with trait values: {sum(~pd.isna(trait_values))}")
print(f"Trait value counts: {trait_values.value_counts().to_dict()}")

# Handle missing values using the library function
processed_df = handle_missing_values(linked_data, trait)
print("Shape after handling missing values:", processed_df.shape)

# Check if any data remains after handling missing values
if processed_df.shape[0] == 0 or processed_df.shape[1] == 0:
    print("WARNING: No data remains after handling missing values.")
    # In this case, we'll set is_trait_biased to True as the dataset is unusable
    is_trait_biased = True
    unbiased_linked_data = processed_df
else:
    # Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(processed_df, trait)

# Conduct quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from eutopic and ectopic endometrial tissues from women with endometriosis."
)

# If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")